### Libraries

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# (Optional) If you're working inside Jupyter
%load_ext autoreload
%autoreload 2
%matplotlib inline



In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Using device: mps


In [3]:
from data_processor_v2 import DataProcessorUpdated

processor = DataProcessorUpdated()

In [4]:
processor.load_and_clean_data()

2023-12-31 23:00:00


/Users/hongyuli/Documents/Classes/Spring2025/NNDL/Projects/repo/energy-prediction/pytorch/data_processor_v2.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[time_col] = pd.to_datetime(df[time_col])


2024-01-01 23:00:00
2023-12-31 23:00:00
2023-12-31 23:00:00


In [5]:
processor.combine_all_data()

In [6]:
processor.df

,DAP_SystemLambda,SCED_system_lambda,Fuel_coal_and_lignite,Fuel_hydro,Fuel_nuclear,Fuel_power_storage,Fuel_solar,Fuel_wind,Fuel_natural_gas,Fuel_other,Load_load,delta_price
2019-01-01 00:00:00,NaN,13.837562,6116.419040,185.465296,3895.951940,0.0,0.0,14311.36445,12512.80815,2.534772,36951.0,NaN
2019-01-01 01:00:00,NaN,15.464869,6423.242360,186.667008,3894.973176,0.0,0.0,14298.52586,12434.13638,2.947464,37112.0,NaN
2019-01-01 02:00:00,NaN,15.487720,6309.280752,187.408832,3894.733152,0.0,0.0,14030.82875,12797.25831,-1.954544,37154.0,NaN
2019-01-01 03:00:00,NaN,15.770092,6416.671292,187.817564,3894.714576,0.0,0.0,13610.13937,13279.01803,1.887324,37283.0,NaN
2019-01-01 04:00:00,NaN,16.036085,6569.580884,186.990116,3892.748912,0.0,0.0,13414.14969,13585.26799,-0.201100,37817.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 19:00:00,23.1651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 20:00:00,23.2113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 21:00:00,21.3244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 22:00:00,20.3351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
processor.shift_dap()

In [8]:
processor.df

,DAP_SystemLambda,SCED_system_lambda,Fuel_coal_and_lignite,Fuel_hydro,Fuel_nuclear,Fuel_power_storage,Fuel_solar,Fuel_wind,Fuel_natural_gas,Fuel_other,Load_load,delta_price
2019-01-01 00:00:00,23.9250,13.837562,6116.419040,185.465296,3895.951940,0.0,0.0,14311.36445,12512.80815,2.534772,36951.0,NaN
2019-01-01 01:00:00,23.3140,15.464869,6423.242360,186.667008,3894.973176,0.0,0.0,14298.52586,12434.13638,2.947464,37112.0,NaN
2019-01-01 02:00:00,23.3475,15.487720,6309.280752,187.408832,3894.733152,0.0,0.0,14030.82875,12797.25831,-1.954544,37154.0,NaN
2019-01-01 03:00:00,23.0595,15.770092,6416.671292,187.817564,3894.714576,0.0,0.0,13610.13937,13279.01803,1.887324,37283.0,NaN
2019-01-01 04:00:00,25.2672,16.036085,6569.580884,186.990116,3892.748912,0.0,0.0,13414.14969,13585.26799,-0.201100,37817.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
processor.split_data(feature_columns=['DAP_SystemLambda', 'SCED_system_lambda', 'Load_load'])

In [10]:
processor.standardize_data()

In [11]:
processor.shift_data()

In [28]:
df, x_train_lstm, y_train_lstm, x_val_lstm, y_val_lstm, x_test_lstm, y_test_lstm = processor.get_data()

In [29]:
x_train_lstm.shape

(26089, 168, 3)

In [30]:
y_train_lstm.shape

(26089, 24, 1)

In [31]:
y_val_lstm.shape

(8401, 24, 1)

In [32]:
y_train_lstm = np.squeeze(y_train_lstm, axis=-1)   # becomes (8401, 24)
y_val_lstm = np.squeeze(y_val_lstm, axis=-1)       # same
y_test_lstm = np.squeeze(y_test_lstm, axis=-1)

# Model Building

In [33]:
from LSTM import LSTMModel  # assuming you saved your class as LSTM.py

# Example data shapes
input_size = x_train_lstm.shape[2]   # number of features per timestep (e.g., 5)
output_size = y_train_lstm.shape[1]  # number of output values (e.g., 24 hours)

# Create model
new_model = LSTMModel(
    input_size=input_size,
    hidden_size=512,          # you can adjust, 512 is a good start
    output_size=output_size,
    num_layers=2,             # deeper model (recommend 2 layers for LSTM)
    dropout=0.1               # optional regularization
)

new_model.to(device)


LSTMModel(
  (lstm): LSTM(3, 512, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=512, out_features=24, bias=True)
)

In [35]:
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss  # or define it above in your notebook

trainer = ModelTrainer(
    model=new_model,
    features_training_data=x_train_lstm,
    target_training_data=y_train_lstm,
    features_eval_data=x_val_lstm,
    target_eval_data=y_val_lstm,
    device=device,
    loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=0.2)
)


trainer.train(epochs=100, batch_size=32, patience=50, learning_rate=5e-4)


RuntimeError: Invalid buffer size: 21.54 GB

In [ ]:
import optuna
from MLP import MLPModel
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss

def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    n_layers = trial.suggest_int("n_layers", 2, 4)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    alpha = trial.suggest_uniform("alpha", 0.1, 0.5)

    # Build hidden layers
    hidden_layers = [hidden_dim] * n_layers

    # Build model
    model = MLPModel(input_shape=len(x_train_lstm[1]), output_shape=len(y_train_lstm[1]), hidden_layers=hidden_layers).get_model().to(device)

    # Trainer
    trainer = ModelTrainer(
        model=model,
        features_training_data=x_train_lstm,
        target_training_data=y_train_lstm,
        features_eval_data=x_val_lstm,
        target_eval_data=y_val_lstm,
        device=device,
        loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=alpha)
    )

    # Train for a few epochs
    trainer.train(epochs=50, batch_size=32, patience=10, learning_rate=lr)

    # Return final eval loss (or early stopping best)
    return trainer.history['eval_loss'][-1]


In [115]:

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner()
)
study.optimize(objective, n_trials=50)


[I 2025-05-09 16:42:11,678] A new study created in memory with name: no-name-a698c28b-2645-47e5-9c4e-20eeea0fe031
/var/folders/wd/tgznb8111md97qlw9yqd793m0000gn/T/ipykernel_93659/1319189539.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
/var/folders/wd/tgznb8111md97qlw9yqd793m0000gn/T/ipykernel_93659/1319189539.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.1, 0.5)


Epoch 1: Train Loss = 0.9701, Eval Loss = 0.2825
Epoch 2: Train Loss = 0.6931, Eval Loss = 0.2827
Epoch 3: Train Loss = 0.6292, Eval Loss = 0.2827
Epoch 4: Train Loss = 0.6035, Eval Loss = 0.2810
Epoch 5: Train Loss = 0.5935, Eval Loss = 0.2807
Epoch 6: Train Loss = 0.5770, Eval Loss = 0.2806
Epoch 7: Train Loss = 0.5642, Eval Loss = 0.2794
Epoch 8: Train Loss = 0.5514, Eval Loss = 0.2780
Epoch 9: Train Loss = 0.5346, Eval Loss = 0.2767
Epoch 10: Train Loss = 0.5215, Eval Loss = 0.2763
Epoch 11: Train Loss = 0.5073, Eval Loss = 0.2757
Epoch 12: Train Loss = 0.4906, Eval Loss = 0.2751
Epoch 13: Train Loss = 0.4776, Eval Loss = 0.2741
Epoch 14: Train Loss = 0.4626, Eval Loss = 0.2739
Epoch 15: Train Loss = 0.4611, Eval Loss = 0.2732
Epoch 16: Train Loss = 0.4435, Eval Loss = 0.2738
Epoch 17: Train Loss = 0.4321, Eval Loss = 0.2729
Epoch 18: Train Loss = 0.4210, Eval Loss = 0.2731
Epoch 19: Train Loss = 0.4135, Eval Loss = 0.2729
Epoch 20: Train Loss = 0.4025, Eval Loss = 0.2733
Epoch 21:

[I 2025-05-09 16:44:23,971] Trial 0 finished with value: 0.27223822474479675 and parameters: {'hidden_dim': 512, 'n_layers': 3, 'lr': 1.852800155011604e-05, 'alpha': 0.33719711995218815}. Best is trial 0 with value: 0.27223822474479675.


Epoch 50: Train Loss = 0.2736, Eval Loss = 0.2722
Epoch 1: Train Loss = 1.1218, Eval Loss = 0.2850
Epoch 2: Train Loss = 0.9999, Eval Loss = 0.2813
Epoch 3: Train Loss = 0.8356, Eval Loss = 0.2798
Epoch 4: Train Loss = 0.7409, Eval Loss = 0.2799
Epoch 5: Train Loss = 0.6869, Eval Loss = 0.2799
Epoch 6: Train Loss = 0.6520, Eval Loss = 0.2802
Epoch 7: Train Loss = 0.6312, Eval Loss = 0.2803
Epoch 8: Train Loss = 0.6182, Eval Loss = 0.2803
Epoch 9: Train Loss = 0.6086, Eval Loss = 0.2803
Epoch 10: Train Loss = 0.6021, Eval Loss = 0.2803
Epoch 11: Train Loss = 0.5959, Eval Loss = 0.2800
Epoch 12: Train Loss = 0.5928, Eval Loss = 0.2800


[I 2025-05-09 16:44:57,167] Trial 1 finished with value: 0.28002551198005676 and parameters: {'hidden_dim': 128, 'n_layers': 4, 'lr': 1.3594728164092837e-05, 'alpha': 0.3120330930570845}. Best is trial 0 with value: 0.27223822474479675.


...

Epoch 35: Train Loss = 0.3907, Eval Loss = 0.2946
Early stopping triggered at epoch 35
Epoch 1: Train Loss = 0.8270, Eval Loss = 0.2425
Epoch 2: Train Loss = 0.5939, Eval Loss = 0.2425
Epoch 3: Train Loss = 0.5274, Eval Loss = 0.2409
Epoch 4: Train Loss = 0.4949, Eval Loss = 0.2418
Epoch 5: Train Loss = 0.4711, Eval Loss = 0.2406
Epoch 6: Train Loss = 0.4523, Eval Loss = 0.2392
Epoch 7: Train Loss = 0.4341, Eval Loss = 0.2386
Epoch 8: Train Loss = 0.4192, Eval Loss = 0.2383
Epoch 9: Train Loss = 0.4053, Eval Loss = 0.2369
Epoch 10: Train Loss = 0.3898, Eval Loss = 0.2370
Epoch 11: Train Loss = 0.3740, Eval Loss = 0.2367
Epoch 12: Train Loss = 0.3643, Eval Loss = 0.2368
Epoch 13: Train Loss = 0.3504, Eval Loss = 0.2368
Epoch 14: Train Loss = 0.3397, Eval Loss = 0.2363
Epoch 15: Train Loss = 0.3296, Eval Loss = 0.2365
Epoch 16: Train Loss = 0.3198, Eval Loss = 0.2363
Epoch 17: Train Loss = 0.3106, Eval Loss = 0.2364
Epoch 18: Train Loss = 0.3024, Eval Loss = 0.2366
Epoch 19: Train Loss =

[I 2025-05-10 11:45:07,422] Trial 48 finished with value: 0.2368868887424469 and parameters: {'hidden_dim': 512, 'n_layers': 2, 'lr': 2.4588115735124388e-05, 'alpha': 0.11106523222120268}. Best is trial 48 with value: 0.2368868887424469.


Epoch 24: Train Loss = 0.2590, Eval Loss = 0.2369
Early stopping triggered at epoch 24
Epoch 1: Train Loss = 1.0233, Eval Loss = 0.2851
Epoch 2: Train Loss = 0.7906, Eval Loss = 0.2830
Epoch 3: Train Loss = 0.6898, Eval Loss = 0.2822
Epoch 4: Train Loss = 0.6429, Eval Loss = 0.2822
Epoch 5: Train Loss = 0.6176, Eval Loss = 0.2822
Epoch 6: Train Loss = 0.6020, Eval Loss = 0.2817
Epoch 7: Train Loss = 0.5860, Eval Loss = 0.2815
Epoch 8: Train Loss = 0.5773, Eval Loss = 0.2810
Epoch 9: Train Loss = 0.5657, Eval Loss = 0.2799
Epoch 10: Train Loss = 0.5560, Eval Loss = 0.2798
Epoch 11: Train Loss = 0.5473, Eval Loss = 0.2790
Epoch 12: Train Loss = 0.5376, Eval Loss = 0.2789
Epoch 13: Train Loss = 0.5376, Eval Loss = 0.2786
Epoch 14: Train Loss = 0.5288, Eval Loss = 0.2784
Epoch 15: Train Loss = 0.5106, Eval Loss = 0.2777
Epoch 16: Train Loss = 0.5014, Eval Loss = 0.2776
Epoch 17: Train Loss = 0.4923, Eval Loss = 0.2770
Epoch 18: Train Loss = 0.4830, Eval Loss = 0.2765
Epoch 19: Train Loss =

[I 2025-05-10 11:46:17,847] Trial 49 finished with value: 0.276837021112442 and parameters: {'hidden_dim': 256, 'n_layers': 2, 'lr': 2.4559785909044423e-05, 'alpha': 0.3396992502904647}. Best is trial 48 with value: 0.2368868887424469.


Epoch 36: Train Loss = 0.3725, Eval Loss = 0.2768
Early stopping triggered at epoch 36


In [ ]:
print("Best hyperparameters:", study.best_trial.params)


In [ ]:
# predict

mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(torch.tensor(x_test_mlp, dtype=torch.float32).to(device))


y_pred = y_pred.cpu().numpy()
errors = y_pred - y_test_mlp  # assuming y_test is numpy array
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")



Test MAE: 44.8259
Test RMSE: 257.4293
